# NLP Document Locator

## Import statements

In [1]:
import os
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

import pandas as pd
import numpy as np
import random
from datetime import datetime

[nltk_data] Downloading package punkt to C:\Users\Hitesh
[nltk_data]     Somani\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Hitesh
[nltk_data]     Somani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Load data

In [2]:
import json
with open(os.path.join(os.getcwd(), 'data', 'train2.json')) as file:
    data = json.load(file)

## Preprocessing

In [3]:
# words = []
# labels = []
# docs_x = []
# docs_y = []

# # ------------------- TOKENIZE ----------------- #
# for intent in data['intents']:
#     for pattern in intent['patterns']:
#         wrds = nltk.word_tokenize(pattern)
#         words.extend(wrds)
#         docs_x.append(wrds)
#         docs_y.append(intent["tag"])
        
#     if intent['tag'] not in labels:
#         labels.append(intent['tag'])
       

In [4]:
# # --------------------STOP WORD REMOVAL ----------#
# from nltk.corpus import stopwords
# stop_words = list(set(stopwords.words('english'))) 

# for word in words:
#     if word in stop_words:
#         words.remove(word)

# for wrd in docs_x:
#     for word in wrd:
#         if word in stop_words:
#             wrd.remove(word)

In [5]:
# # ------------------ REMOVE PUNCTUATION ----------- #
# import string
# for word in words:
#     word = word.translate(str.maketrans('', '', string.punctuation))
#     word = word.strip()


# for wrd in docs_x:
#     for word in wrd:
#         word = word.translate(str.maketrans('', '', string.punctuation))
#         word = word.strip()

In [6]:
# # ---------------- STEMMING ------------------ #
# words = [stemmer.stem(w.lower()) for w in words if w != "?"]
# words = sorted(list(set(words)))

# labels = sorted(labels)

In [7]:
rows = []
labels = []
for intent in data['intents']:
    rows.extend(intent['train_data'])
    label = []
    label.append(intent['label'])
    labels.extend(label*len(intent['train_data']))

### Date Recognition

In [8]:
from datetime import datetime

def decode_date(tokens):

    months_fullform = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'decemeber']
    months_shortform = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    month = 0
    for token in tokens:
        try:
            int(token)
        except:
            if token.lower() in months_fullform:
                month = months_fullform.index(token.lower()) + 1
            elif token.lower() in months_shortform:
                month = months_shortform.index(token.lower()) + 1
            tokens.remove(token)
        
    if len(tokens) > 2:
        if month:
            day = int(tokens[0])
            year = int(tokens[2])
        else:
            day = int(tokens[0])
            month = int(tokens[1])
            year = int(tokens[2])
    elif len(tokens) > 1:
        if month:
            day = int(tokens[0])
            year = 2020
        else:
            day = int(tokens[0])
            month = int(tokens[1])
            year = 2020
    elif len(tokens)==1:
        day = int(tokens[0])
        year = 2020
    return (day, month, year)

In [9]:
# !python3 -m spacy download en_core_web_lg
import spacy
sp_lg = spacy.load('en_core_web_lg')
import nltk
import re

query_date = {}
for idx, row in enumerate(rows):
    date_occurences = [(ent.text.strip(), ent.label_) for ent in sp_lg(row).ents if ent.label_ == 'DATE']
    query_date[row] = []
    
    for date in date_occurences:
        try:
            date_token = re.split('\s+|/|-|:',date[0])
            day, month, year = decode_date(date_token)
            query_date[row].append((f'{year}-{month}-{day}'))
            row = row.replace(date[0], "", 1)
            rows[idx] = row
        except:
            pass

### Universal Sentence Encoder

In [10]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [11]:
embedding = embed(rows)

X = pd.DataFrame(data=embedding.numpy())
X['label'] = labels

In [12]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,label
0,-0.004851,-0.041964,-0.051808,0.019784,0.021853,0.021047,0.043125,-0.001204,0.022696,-0.077722,...,0.039908,0.052083,0.001058,0.065795,0.020701,-0.016375,-0.004406,-0.080554,-0.008480,marketing_google-home
1,0.010735,0.001598,-0.069327,0.045384,0.010653,0.049769,0.043490,0.014172,0.046780,-0.072827,...,-0.025100,0.028380,-0.019510,0.065093,0.041348,-0.048325,0.043596,-0.082165,0.022102,marketing_google-home
2,0.008150,-0.025516,-0.051825,0.018839,-0.011497,0.000663,0.038303,0.017663,0.018018,-0.060026,...,0.040483,0.025089,-0.002037,0.066990,0.038984,-0.020493,0.034859,-0.082567,-0.016747,marketing_google-home
3,-0.002875,0.017606,-0.065296,0.043215,-0.054827,0.022873,-0.023303,0.021275,0.003382,-0.069676,...,-0.022157,0.063531,0.023405,0.070117,0.007039,-0.045061,0.043073,-0.077875,0.045327,marketing_google-home
4,0.001530,-0.010831,-0.072277,0.053805,-0.000521,0.038126,0.050321,0.012032,0.036074,-0.065149,...,0.040641,0.056175,-0.001573,0.071447,0.020559,-0.032644,0.016579,-0.086796,0.013866,marketing_google-home


In [13]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_similarity(labels, features, rotation):
    corr = np.inner(features, features)
    sns.set(font_scale=1)
    fig, ax = plt.subplots(figsize=(18, 18))
    g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd", 
      ax=ax)
    g.set_xticklabels(labels, rotation=rotation)
    g.set_title("Semantic Textual Similarity")

def run_and_plot(messages_):
    message_embeddings_ = embed(messages_)
    plot_similarity(messages_, message_embeddings_, 90)


messages = rows

# run_and_plot(messages)
               

### t-SNE

In [14]:
from sklearn.manifold import TSNE
import time

time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=10, n_iter=300)
tsne_results = tsne.fit_transform(X.drop(columns=['label']))
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

[t-SNE] Computing 31 nearest neighbors...
[t-SNE] Indexed 210 samples in 0.021s...
[t-SNE] Computed neighbors for 210 samples in 0.085s...
[t-SNE] Computed conditional probabilities for sample 210 / 210
[t-SNE] Mean sigma: 0.317312
[t-SNE] KL divergence after 250 iterations with early exaggeration: 59.849529
[t-SNE] KL divergence after 300 iterations: 0.512018
t-SNE done! Time elapsed: 0.5449728965759277 seconds


In [15]:
# tsne_results

In [16]:
tsne_results[0:9, 0]

array([-4.4998074, -4.999609 , -4.691812 , -7.9915156, -4.4400625,
       -5.7924824, -6.1607137, -5.9472117, -7.4384813], dtype=float32)

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns

# fig, ax = plt.subplots(figsize=(12,9))
# ax.scatter(tsne_results[0:18, 0], tsne_results[0:18, 1])
# plt.show()

df_subset = pd.DataFrame()
df_subset['tsne-2d-one'] = tsne_results[:,0]
df_subset['tsne-2d-two'] = tsne_results[:,1]
df_subset['label'] = X['label']

In [18]:
type(df_subset)

pandas.core.frame.DataFrame

In [20]:
plt.figure(figsize=(26,16))
sns.scatterplot(
    x="tsne-2d-one", 
    y="tsne-2d-two",
    hue="label",
    palette=sns.color_palette("hls", 14),
    data=df_subset,
    legend="full",
    alpha=1,
    s=100
)

ValueError: zero-size array to reduction operation minimum which has no identity

<Figure size 1872x1152 with 1 Axes>

## Modelling

In [24]:
# Data Prepraration
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

y = X['label']
X = X.drop(columns=['label'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=10)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [25]:
set(X_train.index).intersection(set(X_test.index))

set()

So an empty intersection set of both sets proves that there is no observation common in train and test data

In [26]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [27]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [28]:
# Building Neural network
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model = Sequential()
model.add(Dense(64, input_shape=(512,)))
model.add(Dense(100))
model.add(Dense(1000))
model.add(Dense(100))
model.add(Dense(len(set(y_train))))
model.add(Activation('softmax'))

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                32832     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               6500      
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              101000    
_________________________________________________________________
dense_3 (Dense)              (None, 100)               100100    
_________________________________________________________________
dense_4 (Dense)              (None, 14)                1414      
_________________________________________________________________
activation (Activation)      (None, 14)                0         
Total params: 241,846
Trainable params: 241,846
Non-trainable params: 0
__________________________________________________

In [30]:
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [31]:
from tensorflow import keras
keras.utils.plot_model(model)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [32]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_test, y_test))

Epoch 1/10
6/6 [==============================] - 0s 80ms/step - loss: 2.4879 - sparse_categorical_accuracy: 0.3631 - val_loss: 2.2029 - val_sparse_categorical_accuracy: 0.4286
Epoch 2/10
6/6 [==============================] - 0s 11ms/step - loss: 1.7586 - sparse_categorical_accuracy: 0.6071 - val_loss: 1.4365 - val_sparse_categorical_accuracy: 0.6190
Epoch 3/10
6/6 [==============================] - 0s 10ms/step - loss: 0.9833 - sparse_categorical_accuracy: 0.7976 - val_loss: 0.6816 - val_sparse_categorical_accuracy: 0.9048
Epoch 4/10
6/6 [==============================] - 0s 9ms/step - loss: 0.4925 - sparse_categorical_accuracy: 0.8690 - val_loss: 0.3647 - val_sparse_categorical_accuracy: 0.9286
Epoch 5/10
6/6 [==============================] - 0s 8ms/step - loss: 0.3000 - sparse_categorical_accuracy: 0.9048 - val_loss: 0.1650 - val_sparse_categorical_accuracy: 0.9762
Epoch 6/10
6/6 [==============================] - 0s 7ms/step - loss: 0.3175 - sparse_categorical_accuracy: 0.9167 - 

In [33]:
#%tensorboard --logdir logs/fit

In [34]:
# gcloud compute ssh python-nlp-document-finder --project nlp-document-finder --zone us-west1-b -- -L 2222:localhost:6007 

### Prediction 

In [35]:
y_proba = model.predict(X_test)

In [36]:
y_pred = y_proba.argmax(axis=1)

In [37]:
y_proba

array([[4.98759234e-03, 9.84964430e-01, 6.71407724e-06, 9.13460273e-03,
        1.36495655e-05, 3.03940160e-05, 5.27804689e-10, 6.82358863e-04,
        2.37962086e-05, 1.55866059e-04, 3.52326992e-07, 9.56514086e-08,
        2.50314230e-07, 5.66184291e-08],
       [7.22175141e-12, 1.84563864e-10, 6.42649475e-11, 1.14629970e-07,
        5.15991520e-11, 1.84587501e-02, 9.81525898e-01, 1.29534383e-09,
        2.53956280e-13, 1.52937996e-09, 2.01009520e-08, 1.52097018e-05,
        4.28962217e-08, 1.53987188e-11],
       [6.48479230e-12, 2.05469378e-10, 2.28300382e-11, 6.25889811e-08,
        1.74424086e-11, 5.40341530e-03, 9.94575977e-01, 9.24524513e-10,
        4.95492482e-13, 1.39711565e-09, 7.58649144e-09, 2.04415246e-05,
        7.49209192e-08, 3.69173372e-11],
       [8.36413892e-06, 3.61919660e-07, 1.33705202e-09, 4.02204013e-13,
        6.59873156e-10, 3.73988153e-14, 3.39111683e-10, 8.63878413e-10,
        4.05685417e-03, 1.16059193e-04, 9.95816171e-01, 9.99893426e-15,
        3.274

In [38]:
le.classes_

array(['marketing_google-home', 'marketing_kaggle',
       'marketing_marketing apple', 'marketing_marketing_1',
       'marketing_marketing_apple', 'marketing_marketing_copy',
       'marketing_marketing_copy_copy', 'marketing_pixel',
       'research_computer_vision', 'research_nlp',
       'research_quantum_computing', 'sales_ad_words', 'sales_pixel',
       'sales_search_history'], dtype=object)

In [39]:
rows[X_test.index[2]]

'display marketing copy copy marketing document'

In [40]:
from operator import itemgetter
final_results = pd.DataFrame(data = list(itemgetter(*list(X_test.index))(rows)), index=X_test.index, columns=['inputs'])  

In [41]:
final_results['predictions'] = le.inverse_transform(y_pred)
final_results['true'] = le.inverse_transform(y_test)
final_results['query_search'] = ""

In [42]:
final_results

,inputs,predictions,true,query_search
24,please open kaggle marketing report,marketing_kaggle,marketing_kaggle,
91,open marketing copy copy marketing report,marketing_marketing_copy_copy,marketing_marketing_copy_copy,
98,display marketing copy copy marketing document,marketing_marketing_copy_copy,marketing_marketing_copy_copy,
163,display quantum computing research file,research_quantum_computing,research_quantum_computing,
52,show me marketing marketing document,marketing_marketing_1,marketing_marketing_1,
72,show me marketing apple marketing file,marketing_marketing_apple,marketing_marketing_apple,
121,open computer vision research report,research_computer_vision,research_computer_vision,
117,show me pixel marketing file,marketing_pixel,marketing_pixel,
103,display marketing copy copy marketing file,marketing_marketing_copy_copy,marketing_marketing_copy_copy,
106,open pixel marketing report,marketing_pixel,marketing_pixel,


In [43]:
for index in final_results.index:
    for intent in data['intents']:
        if intent['label'] == final_results.loc[index, 'predictions']:
            final_results.loc[index, 'query_search'] = intent['responses']
            print(intent['responses'])

['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\marketing\\kaggle.docx']
['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\marketing\\marketing_copy_copy.docx']
['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\marketing\\marketing_copy_copy.docx']
['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\research\\quantum_computing.pdf']
['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\marketing\\marketing_1.docx']
['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\marketing\\marketing_apple.docx']
['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\research\\computer_vision.pdf']
['C:\\Users\\Hitesh Somani\\Docume

In [44]:
final_results

,inputs,predictions,true,query_search
24,please open kaggle marketing report,marketing_kaggle,marketing_kaggle,[C:\Users\Hitesh Somani\Documents\hobby_projec...
91,open marketing copy copy marketing report,marketing_marketing_copy_copy,marketing_marketing_copy_copy,[C:\Users\Hitesh Somani\Documents\hobby_projec...
98,display marketing copy copy marketing document,marketing_marketing_copy_copy,marketing_marketing_copy_copy,[C:\Users\Hitesh Somani\Documents\hobby_projec...
163,display quantum computing research file,research_quantum_computing,research_quantum_computing,[C:\Users\Hitesh Somani\Documents\hobby_projec...
52,show me marketing marketing document,marketing_marketing_1,marketing_marketing_1,[C:\Users\Hitesh Somani\Documents\hobby_projec...
72,show me marketing apple marketing file,marketing_marketing_apple,marketing_marketing_apple,[C:\Users\Hitesh Somani\Documents\hobby_projec...
121,open computer vision research report,research_computer_vision,research_computer_vision,[C:\Users\Hitesh Somani\Documents\hobby_projec...
117,show me pixel marketing file,marketing_pixel,marketing_pixel,[C:\Users\Hitesh Somani\Documents\hobby_projec...
103,display marketing copy copy marketing file,marketing_marketing_copy_copy,marketing_marketing_copy_copy,[C:\Users\Hitesh Somani\Documents\hobby_projec...
106,open pixel marketing report,marketing_pixel,marketing_pixel,[C:\Users\Hitesh Somani\Documents\hobby_projec...


### Model 2 - KNN

In [45]:
def cosine_similarity(x,y):
    mod_x = (sum(x*x))**0.5
    mod_y = (sum(y*y))**0.5
    
    similarity = np.dot(x,y) / (mod_x*mod_y)
    return similarity

In [46]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

In [47]:
pca = PCA(n_components = 30, random_state=10)
pca.fit(X_train)
X_knn_train = pca.transform(X_train)
X_knn_test = pca.transform(X_test)


knn = KNeighborsClassifier(n_neighbors=3, metric=cosine_similarity)
knn.fit(X_knn_train, y_train)

KNeighborsClassifier(metric=<function cosine_similarity at 0x000001FA347BCEE0>,
                     n_neighbors=3)

In [48]:
sum(pca.explained_variance_ratio_)

0.9706600002245978

In [49]:
y_knn_pred = knn.predict(X_knn_test)

In [50]:
y_knn_pred

array([2, 7, 7, 2, 7, 7, 4, 2, 7, 2, 7, 2, 6, 7, 2, 2, 2, 7, 7, 2, 3, 1,
       0, 4, 4, 2, 2, 1, 7, 7, 1, 7, 2, 2, 1, 7, 7, 7, 4, 2, 2, 4])

In [51]:
y_test

array([ 1,  6,  6, 10,  3,  4,  8,  7,  6,  7,  3,  8,  3,  5, 12,  7, 11,
        5,  3,  8,  8,  0,  1, 12, 12,  9,  1,  2,  3,  1,  0,  0, 13, 11,
        0,  0,  3,  6, 13,  9,  8, 13])

In [52]:
le.inverse_transform(y_knn_pred)

array(['marketing_marketing apple', 'marketing_pixel', 'marketing_pixel',
       'marketing_marketing apple', 'marketing_pixel', 'marketing_pixel',
       'marketing_marketing_apple', 'marketing_marketing apple',
       'marketing_pixel', 'marketing_marketing apple', 'marketing_pixel',
       'marketing_marketing apple', 'marketing_marketing_copy_copy',
       'marketing_pixel', 'marketing_marketing apple',
       'marketing_marketing apple', 'marketing_marketing apple',
       'marketing_pixel', 'marketing_pixel', 'marketing_marketing apple',
       'marketing_marketing_1', 'marketing_kaggle',
       'marketing_google-home', 'marketing_marketing_apple',
       'marketing_marketing_apple', 'marketing_marketing apple',
       'marketing_marketing apple', 'marketing_kaggle', 'marketing_pixel',
       'marketing_pixel', 'marketing_kaggle', 'marketing_pixel',
       'marketing_marketing apple', 'marketing_marketing apple',
       'marketing_kaggle', 'marketing_pixel', 'marketing_pixel',
  

In [53]:
final_results.head()

,inputs,predictions,true,query_search
24,please open kaggle marketing report,marketing_kaggle,marketing_kaggle,[C:\Users\Hitesh Somani\Documents\hobby_projec...
91,open marketing copy copy marketing report,marketing_marketing_copy_copy,marketing_marketing_copy_copy,[C:\Users\Hitesh Somani\Documents\hobby_projec...
98,display marketing copy copy marketing document,marketing_marketing_copy_copy,marketing_marketing_copy_copy,[C:\Users\Hitesh Somani\Documents\hobby_projec...
163,display quantum computing research file,research_quantum_computing,research_quantum_computing,[C:\Users\Hitesh Somani\Documents\hobby_projec...
52,show me marketing marketing document,marketing_marketing_1,marketing_marketing_1,[C:\Users\Hitesh Somani\Documents\hobby_projec...


## User Input Test

In [54]:
le.classes_

array(['marketing_google-home', 'marketing_kaggle',
       'marketing_marketing apple', 'marketing_marketing_1',
       'marketing_marketing_apple', 'marketing_marketing_copy',
       'marketing_marketing_copy_copy', 'marketing_pixel',
       'research_computer_vision', 'research_nlp',
       'research_quantum_computing', 'sales_ad_words', 'sales_pixel',
       'sales_search_history'], dtype=object)

In [55]:
import joblib

joblib.dump(le, 'le.joblib')

['le.joblib']

#### Test1: Asking with document name and folder name

In [56]:
user_input = ["Hi open the sales google home document please"]

embedding = embed(user_input)
X_user = pd.DataFrame(data=embedding.numpy())

In [57]:
y_proba = model.predict(X_user)
y_pred = y_proba.argmax(axis=1)
print(f"label predicted: {le.inverse_transform(y_pred)}")
for intent in data['intents']:
    if intent['label'] == le.inverse_transform(y_pred):
        print(f"Query search: {intent['responses']}")

label predicted: ['marketing_google-home']
Query search: ['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\marketing\\google-home.docx']


In [58]:
(y_proba)

array([[9.7419769e-01, 9.9940415e-05, 1.3899156e-09, 3.3391490e-08,
        7.0754197e-10, 1.1299594e-08, 9.2031987e-06, 3.2007049e-06,
        3.8126229e-05, 1.3764459e-04, 5.3037570e-07, 1.2741656e-03,
        1.1600889e-02, 1.2638419e-02]], dtype=float32)

#### Result: The user made a mistake. google home document is in marketing folder

#### Test2: Asking with document name and folder name but _'can you get me'_ this is not used in training data so far 

In [59]:
user_input = ["Hi can you get me sales pixel document"]

embedding = embed(user_input)
X_user = pd.DataFrame(data=embedding.numpy())

In [60]:
y_proba = model.predict(X_user)
y_pred = y_proba.argmax(axis=1)
print(f"label predicted: {le.inverse_transform(y_pred)}")
for intent in data['intents']:
    if intent['label'] == le.inverse_transform(y_pred):
        print(f"Query search: {intent['responses']}")

label predicted: ['sales_pixel']
Query search: ['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\sales\\pixel.xlsx']


In [61]:
y_proba

array([[7.7512370e-05, 2.3523755e-07, 1.1877052e-10, 5.5037798e-08,
        6.1647423e-11, 1.4119982e-07, 4.6294217e-06, 2.4280565e-04,
        5.3722360e-08, 3.4548917e-10, 2.8916642e-09, 6.5612129e-04,
        9.9893302e-01, 8.5428350e-05]], dtype=float32)

#### Result: The user asked for correct document from correct folder and model answer is correct.

#### Test3: Asking with document name but no folder name

In [62]:
user_input = ["Hi can you get me pixel document"]

embedding = embed(user_input)
X_user = pd.DataFrame(data=embedding.numpy())

In [63]:
y_proba = model.predict(X_user)
y_pred = y_proba.argmax(axis=1)
print(f"label predicted: {le.inverse_transform(y_pred)}")
for intent in data['intents']:
    if intent['label'] == le.inverse_transform(y_pred):
        print(f"Query search: {intent['responses']}")

label predicted: ['sales_pixel']
Query search: ['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\sales\\pixel.xlsx']


#### Result: The user asked for correct document but did not specify and folder. Model gave just one correct result

In [64]:
# Lets y_proba

y_proba

array([[1.27024710e-01, 1.65213412e-03, 1.33802885e-07, 5.09077563e-06,
        8.31018099e-08, 1.24897554e-06, 1.16740575e-05, 5.00009023e-02,
        1.22211420e-03, 4.69118413e-06, 1.11085661e-04, 5.27472439e-05,
        8.17319989e-01, 2.59333337e-03]], dtype=float32)

So here is one room for improvement. The model should also give high probability to second element of the above list sonce that corresponds to 'marketing_pixel' <br>
May be we can add few more such statements in training data itself.

#### Test4: Asking document created on a date

In [65]:
user_input = ["Please open kaggle marketing document created on 15 July"]

In [66]:
query_date = {}
for idx, row in enumerate(user_input):
    date_occurences = [(ent.text.strip(), ent.label_) for ent in sp_lg(row).ents if ent.label_ == 'DATE']
    query_date[row] = []
    
    for date in date_occurences:
        try:
            date_token = re.split('\s+|/|-|:',date[0])
            day, month, year = decode_date(date_token)
            query_date[row].append((day, month, year))
            row = row.replace(date[0], "", 1)
            user_input[idx] = row
        except:
            pass

In [67]:
embedding = embed(user_input)
X_user = pd.DataFrame(data=embedding.numpy())

In [68]:
y_proba = model.predict(X_user)
y_pred = y_proba.argmax(axis=1)
print(f"label predicted: {le.inverse_transform(y_pred)}")
for intent in data['intents']:
    if intent['label'] == le.inverse_transform(y_pred):
        print(f"Query search: {intent['responses']}")

label predicted: ['marketing_kaggle']
Query search: ['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\marketing\\kaggle.docx']


In [69]:
# Lets y_proba

y_proba

array([[1.7530941e-03, 9.8977262e-01, 2.9418918e-05, 5.1290165e-03,
        6.3402244e-05, 5.8071248e-05, 7.9478282e-09, 1.5068757e-04,
        1.4374295e-04, 2.8812897e-03, 1.8720624e-05, 2.0043016e-08,
        1.8837184e-08, 5.6920868e-08]], dtype=float32)

In [70]:
query_date

{'Please open kaggle marketing document created on 15 July': [(15, 7, 2020)]}

#### Test5: Asking document created on a date

In [71]:
user_input = ["Please open computer vision document created on 1/6/2019", "Please open computer vision document created on 1-6-2019", 
              "Give me google home document from 14/07", "Give me google home document from 14-07", "Give me google home document made 14 Jan", 
              "Give me nlp document made on March 12", "Give me quantum computing document"]

In [72]:
query_date = {}
for idx, row in enumerate(user_input):
    date_occurences = [(ent.text.strip(), ent.label_) for ent in sp_lg(row).ents if ent.label_ == 'DATE']
    query_date[row] = []
    for date in date_occurences:
        try:
            date_token = re.split('\s+|/|-|:',date[0])
            day, month, year = decode_date(date_token)
            query_date[row].append((f'{year}-{month}-{day}'))
            datetime.strptime(f'{year}-{month}-{day}', '%Y-%m-%d %H:%M:%S.%f')
            row = row.replace(date[0], "", 1)
            user_input[idx] = row
        except:
            pass

In [73]:
embedding = embed(user_input)
X_user = pd.DataFrame(data=embedding.numpy())

In [74]:
y_proba = model.predict(X_user)
y_pred = y_proba.argmax(axis=1)

In [75]:
y_pred

array([ 8,  8,  0,  0,  0,  9, 10], dtype=int64)

In [76]:
y_proba = model.predict(X_user)
y_pred = y_proba.argmax(axis=1)
print(f"label predicted: {le.inverse_transform(y_pred)}")
for pred in y_pred:
    for intent in data['intents']:
        if intent['label'] == le.inverse_transform(pred.reshape(1,)):
            print(f"Query search: {intent['responses']}")

label predicted: ['research_computer_vision' 'research_computer_vision'
 'marketing_google-home' 'marketing_google-home' 'marketing_google-home'
 'research_nlp' 'research_quantum_computing']
Query search: ['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\research\\computer_vision.pdf']
Query search: ['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\research\\computer_vision.pdf']
Query search: ['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\marketing\\google-home.docx']
Query search: ['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\marketing\\google-home.docx']
Query search: ['C:\\Users\\Hitesh Somani\\Documents\\hobby_projects\\nlp_document_finder\\nlp_document_finder\\Google\\marketing\\google-home.docx']
Query search: ['C:\\Users\\Hitesh Somani\\Documents\\hobby_p

In [77]:
# Lets y_proba
y_proba

array([[2.9816873e-02, 1.9804962e-02, 1.3232867e-05, 4.6698625e-07,
        1.4549284e-05, 3.6122857e-10, 6.1213795e-10, 2.5969573e-05,
        9.4384402e-01, 3.3179042e-03, 2.4989371e-03, 5.6104621e-10,
        5.3394660e-07, 6.6244457e-04],
       [2.9816873e-02, 1.9804962e-02, 1.3232867e-05, 4.6698625e-07,
        1.4549284e-05, 3.6122857e-10, 6.1213795e-10, 2.5969573e-05,
        9.4384402e-01, 3.3179042e-03, 2.4989371e-03, 5.6104621e-10,
        5.3394660e-07, 6.6244457e-04],
       [9.9573684e-01, 9.0964470e-04, 1.1575663e-09, 3.2863962e-10,
        9.1162194e-10, 1.8923493e-12, 6.2184737e-11, 1.4608567e-07,
        2.1484205e-03, 1.1112575e-03, 8.4510459e-05, 2.4287603e-11,
        2.6839801e-08, 9.2856026e-06],
       [9.9573684e-01, 9.0964470e-04, 1.1575663e-09, 3.2863962e-10,
        9.1162194e-10, 1.8923493e-12, 6.2184737e-11, 1.4608567e-07,
        2.1484205e-03, 1.1112575e-03, 8.4510459e-05, 2.4287603e-11,
        2.6839801e-08, 9.2856026e-06],
       [9.9558699e-01, 2.556

In [78]:
query_date

{'Please open computer vision document created on 1/6/2019': [],
 'Please open computer vision document created on 1-6-2019': ['2019-6-1'],
 'Give me google home document from 14/07': [],
 'Give me google home document from 14-07': ['2020-7-14'],
 'Give me google home document made 14 Jan': ['2020-1-14'],
 'Give me nlp document made on March 12': ['2020-3-12'],
 'Give me quantum computing document': []}

### Save model

In [79]:
model.save('model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets
